In [6]:
%%cmd
pip install hca

Microsoft Windows [Version 6.1.7601]
Copyright (c) 2009 Microsoft Corporation.  All rights reserved.

C:\Users\Justin\Desktop>pip install hca

C:\Users\Justin\Desktop>

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [24]:
#!/usr/bin/env python3

import hca, json

def count_cell_types(cell_type):
    count = 0
    query = { "query": { "match": { "bundle.creator_uid": 0 } } }
    
    # Iterate through all search results containing the chosen cell type
    for result in hca.dss.DSSClient().post_search.iterate(es_query={}, replica='aws', output_format='raw'):
#        print(result)
#        bundle_uuid = result['bundle_fqid'][:36]
#        bundle_dict = hca.dss.DSSClient().get_bundle(uuid=bundle_uuid, replica='aws')
#        print()
        print(json.dumps(result, indent=4, sort_keys=True))
#        print()
#        print(json.dumps(hca.dss.DSSClient().get_file(uuid=bundle_dict.get('bundle').get('files')[0].get('uuid'), replica='aws'), indent=4, sort_keys=True))
        break
        
        # Add up the total number of cells in the sample
#        count += bundle.get('biomaterials')[0].get('content').get('total_estimated_cells')
    return count

def main():
    print(count_cell_types('spleen'))

main()

{
    "bundle_fqid": "0fd98881-8879-4928-939d-e3583ae0d006.2018-05-01T070337.608744Z",
    "bundle_url": "https://dss.data.humancellatlas.org/v1/bundles/0fd98881-8879-4928-939d-e3583ae0d006?version=2018-05-01T070337.608744Z&replica=aws",
    "metadata": {
        "files": {},
        "manifest": {
            "creator_uid": 0,
            "files": [
                {
                    "content-type": "application/json",
                    "crc32c": "003ad03e",
                    "indexed": true,
                    "name": "cell.json",
                    "s3-etag": "fb5aac9f8f0a9da710998cac814a33dc",
                    "sha1": "6e1de4809aa92d3e142830fbcf353c2e03556335",
                    "sha256": "cb718e73527242a757431cbbc94b40878e201608064b77ca6b9ebf900c3c9ee6",
                    "size": 42,
                    "uuid": "ff17b618-e94e-4c0a-a41a-f339ce917358",
                    "version": "2018-05-01T070332.393227Z"
                },
                {
                    "

In [ ]:
%%cmd
pip freeze

In [74]:
client = hca.dss.DSSClient()
client.host

'https://dss.data.humancellatlas.org/v1'